1. 長文のemployee_reviewをサマリー
2. 弱みを識別
3. 弱みを克服するパーソナライズされたプランを作成
の3つをSequentialChainで連続実行する

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore

In [16]:
from langchain.prompts import PromptTemplate

In [17]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [18]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = PromptTemplate.from_template(template1)
# prompt1 = ChatPromptTemplate.from_template(template1)
# chain_1 = LLMChain(llm=llm,
#                      prompt=prompt1,
#                      output_key="review_summary")
# chain_1 = prompt1 | llm
                     

In [19]:
template2 = "Identify key employee weaknesses in this review summary:\n{review_summary}"
prompt2 = PromptTemplate.from_template(template2)
# prompt2 = ChatPromptTemplate.from_template(template2)
# chain_2 = LLMChain(llm=llm,
#                      prompt=prompt2,
#                      output_key="weaknesses")
# chain_2 = prompt2 | llm

In [20]:
template3 = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
prompt3 = PromptTemplate.from_template(template3)
# prompt3 = ChatPromptTemplate.from_template(template3)
# chain_3 = LLMChain(llm=llm,
#                      prompt=prompt3,
#                      output_key="final_plan")
# chain_3 = prompt3 | llm

In [8]:
# chain = (
    
#     chain_1
#     | chain_2
#     | chain_3
# )

In [9]:
# chain.output_schema.schema()

In [47]:
# results = chain.invoke(employee_review)
results = chain.invoke({"review":employee_review})

In [48]:
print(results)
print(type(results))
print(results.content)

content="## Personalized Plan for Joe:\n\n**1. Time Management:**\n\n* **Implement a time-tracking system:** Use a tool like Toggl or Asana to track time spent on tasks and identify time-consuming areas.\n* **Prioritize tasks:** Use a system like Eisenhower Matrix to prioritize tasks based on urgency and importance.\n* **Break down projects:** Divide large projects into smaller, manageable tasks with clear deadlines.\n* **Regular check-ins:** Schedule weekly check-ins with his manager to discuss progress and identify any roadblocks.\n\n**2. Written Communication:**\n\n* **Seek feedback:** Ask colleagues to review his written work and provide constructive feedback.\n* **Utilize online resources:** Explore online resources like Grammarly or Hemingway Editor to improve clarity and conciseness.\n* **Attend workshops:** Consider attending workshops on effective business writing and communication.\n* **Practice regularly:**  Write regularly, whether it's emails, reports, or even short summar

In [21]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [12]:

first_chain = prompt1 | llm | StrOutputParser()
second_chain = prompt2 | llm | StrOutputParser()
third_chain = prompt3 | llm | StrOutputParser()

In [ ]:
complete_chain = ( {
  
})

In [13]:
complete_chain = ({
        "review": itemgetter("review"),
        "review_summary": first_chain,#itemgetter("review_summary"),
        "weaknesses" : second_chain#itemgetter("weaknesses")
        # "weaknesses": itemgetter("weaknesses")
        } 
    | RunnablePassthrough.assign(mysecond=second_chain,mythird=third_chain)
    )

complete_chain.invoke({'review': employee_review})

KeyError: "Input to PromptTemplate is missing variables {'review_summary'}.  Expected: ['review_summary'] Received: ['review']"

In [22]:
model_parser = llm | StrOutputParser() 

In [30]:
chaina = ({"review":RunnablePassthrough()}|prompt1|{"review_summary" : model_parser})


In [39]:
# chainb = prompt2 | model_parser

chainb = ({"review_summary":RunnablePassthrough()}|prompt2|{"weaknesses" : model_parser})




In [40]:
# chainc = prompt3 | model_parser
chainc = prompt3 | model_parser
chainc = ({"weaknesses":RunnablePassthrough()}|prompt3|model_parser)

In [47]:
# qg = (chaina | {"weaknesses":chainb} | prompt3)
qg = (chaina | chainb | chainc)

In [48]:
result = qg.invoke(employee_review)

In [49]:
print(type(result))

<class 'str'>


In [51]:
print(result)

## Personalized Plan to Address Joe's Weaknesses

This plan focuses on addressing Joe's weaknesses in time management, written communication, and delegation, with actionable steps and resources to support his growth.

**1. Time Management:**

* **Goal:** Improve time management skills and minimize missed deadlines.
* **Action Steps:**
    * **Prioritization Techniques:** Learn and implement time management frameworks like Eisenhower Matrix, Pomodoro Technique, or Getting Things Done (GTD).
    * **Task Tracking:** Utilize tools like project management software (Asana, Trello), digital calendars, or to-do list apps to track progress and deadlines.
    * **Time Blocking:** Allocate dedicated time slots for specific tasks to ensure focus and avoid distractions.
    * **Delegate Effectively:** Identify tasks that can be delegated to others, freeing up time for more critical responsibilities.
* **Resources:**
    * **Books:** "Getting Things Done" by David Allen, "The 7 Habits of Highly Eff